<a href="https://colab.research.google.com/github/AloraTab/aat2000/blob/main/dissy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%pip install pandas==1.5.3
%pip install tensorflow-addons==0.19.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 97.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.9 MB/s eta 0:00:00


In [2]:
from tensorflow import keras
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.layers import Attention
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers.schedules import ExponentialDecay, CosineDecayRestarts
from keras.callbacks import Callback
from keras import backend as K

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math

## Imported Classes

In [3]:
# Cosine Annealing Scheduler
# https://github.com/4uiiurz1/keras-cosine-annealing/blob/master/cosine_annealing.py

class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [4]:
# LR Finder in Keras
# https://github.com/avanwyk/tensorflow-projects/tree/master/lr-finder

class LRFinder(Callback):
    """`Callback` that exponentially adjusts the learning rate after each training batch between `start_lr` and
    `end_lr` for a maximum number of batches: `max_step`. The loss and learning rate are recorded at each step allowing
    visually finding a good learning rate as per https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html via
    the `plot` method.
    """

    def __init__(self, start_lr: float = 1e-7, end_lr: float = 10, max_steps: int = 100, smoothing=0.9):
        super(LRFinder, self).__init__()
        self.start_lr, self.end_lr = start_lr, end_lr
        self.max_steps = max_steps
        self.smoothing = smoothing
        self.step, self.best_loss, self.avg_loss, self.lr = 0, 0, 0, 0
        self.lrs, self.losses = [], []

    def on_train_begin(self, logs=None):
        self.step, self.best_loss, self.avg_loss, self.lr = 0, 0, 0, 0
        self.lrs, self.losses = [], []

    def on_train_batch_begin(self, batch, logs=None):
        self.lr = self.exp_annealing(self.step)
        tf.keras.backend.set_value(self.model.optimizer.lr, self.lr)

    def on_train_batch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        step = self.step
        if loss:
            self.avg_loss = self.smoothing * self.avg_loss + (1 - self.smoothing) * loss
            smooth_loss = self.avg_loss / (1 - self.smoothing ** (self.step + 1))
            self.losses.append(smooth_loss)
            self.lrs.append(self.lr)

            if step == 0 or loss < self.best_loss:
                self.best_loss = loss

            if smooth_loss > 4 * self.best_loss or tf.math.is_nan(smooth_loss):
                self.model.stop_training = True

        if step == self.max_steps:
            self.model.stop_training = True

        self.step += 1

    def exp_annealing(self, step):
        return self.start_lr * (self.end_lr / self.start_lr) ** (step * 1. / self.max_steps)

    def plot(self):
        fig, ax = plt.subplots(1, 1)
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate (log scale)')
        ax.set_xscale('log')
        ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.0e'))
        ax.plot(self.lrs, self.losses)

In [5]:
#https://github.com/nathanhubens/KerasOneCycle
from keras.callbacks import Callback
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

#####################
##### OneCycle  #####
#####################

          
class OneCycle(Callback):
    """This callback implements a cyclical learning rate and momentum policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration 
    For more detail, please see paper.
    
    # Example
        ```python
            clr = OneCycle(min_lr=1e-3, max_lr=1e-2,
                      min_mtm=0.85, max_mtm=0.95,
                      annealing=0.1,step_size=np.ceil((X_train.shape[0]*epochs/batch_size)))
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    # Arguments
        min_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - min_lr).
        step_size: number of training iterations in the cycle. To define as `np.ceil((X_train.shape[0]*epochs/batch_size))`
        max_mtm : initial value of the momentum    
        min_mtm : lower boundary in the cycle.
        annealing_stage : percentage of the iterations where the lr
                    will decrease lower than its min_lr
        annealing_rate : in annealing phase learning rate will be decreased to annealing_rate*min_lr
                    
        # References
        Original paper: https://arxiv.org/pdf/1803.09820.pdf
        Inspired by : https://sgugger.github.io/the-1cycle-policy.html#the-1cycle-policy
    """

    def __init__(self, min_lr=1e-5, max_lr=1e-2, min_mtm = 0.85, max_mtm=0.95, training_iterations=1000.,
                 annealing_stage=0.1, annealing_rate=0.01):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.min_mtm = min_mtm
        self.max_mtm = max_mtm
        self.annealing_stage = annealing_stage
        self.step_size = training_iterations*(1-self.annealing_stage)/2
        self.min_annealing_lr = annealing_rate * min_lr
        self.iterations = 0.
        self.training_iterations = training_iterations
        self.history = {}
        
    def clr(self):
        if self.iterations < 2*self.step_size :
            x = np.abs(self.iterations/self.step_size - 1)
            return self.min_lr + (self.max_lr-self.min_lr)*(1-x)
        else:
            x = min(1, float(self.iterations - 2 * self.step_size) / (self.training_iterations - 2 * self.step_size))
            return self.min_lr - (self.min_lr - self.min_annealing_lr) * x
        
    
    def cmtm(self):
        if self.iterations < 2*self.step_size :   
            x = np.abs(self.iterations/self.step_size - 1)
        else: 
            x=1
        return self.min_mtm + (self.max_mtm-self.min_mtm)*(x)     
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        K.set_value(self.model.optimizer.momentum, self.max_mtm)
         
    def on_batch_end(self, batch, logs=None):
        
        logs = logs or {}
        self.iterations += 1
    
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('momentum', []).append(K.get_value(self.model.optimizer.momentum))
        self.history.setdefault('iterations', []).append(self.iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr()) 
        K.set_value(self.model.optimizer.momentum, self.cmtm())
        
    def plot_lr(self):
        plt.xlabel('Training Iterations')
        plt.ylabel('Learning Rate')
        plt.title("CLR - '1 cycle' Policy")
        plt.plot(self.history['iterations'], self.history['lr'])
        
    def plot_mtm(self):
        plt.xlabel('Training Iterations')
        plt.ylabel('Momentum')
        plt.title("CLR - '1 cycle' Policy")
        plt.plot(self.history['iterations'], self.history['momentum'])

# Preparing Dataset

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
PATH = '/content/gdrive/MyDrive/Colab Notebooks/Dataset'
fileName = 'csi-dataset-20-7-5-newpreprocwfil.pkl'
fullPath = os.path.join(PATH, fileName)
print(fullPath)

/content/gdrive/MyDrive/Colab Notebooks/Dataset/csi-dataset-20-7-5-newpreprocwfil.pkl


In [8]:
df = pd.read_pickle(fullPath)

In [18]:
# df = df.sample(frac=1)
Y = df['Label'].values
X = df['Sample'].values
X = [[np.asarray(sample) for sample in i] for i in X]
X = np.asarray(X).astype('float32')

In [19]:
X.shape

(6642, 256, 2, 3, 30)

In [11]:
# unique, counts = np.unique(Y_test, return_counts=True)
# print(np.asarray((unique, counts)).T)

In [12]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
vectorized_y = np_utils.to_categorical(encoded_Y)

In [13]:
classes = encoder.classes_
print(classes)

['I1' 'I11' 'I13' 'I3' 'I5' 'I7' 'I9']


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X,vectorized_y, test_size=0.3, random_state=40)

In [17]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],X_train.shape[2],1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],X_test.shape[2],1))

X_train = tf.convert_to_tensor(X_train)
Y_train = tf.convert_to_tensor(Y_train)

ValueError: ignored

# Building Model

### Custom Components

In [ ]:
# Residual Connections
# https://stackoverflow.com/questions/64792460/how-to-code-a-residual-block-using-two-layers-of-a-basic-cnn-algorithm-built-wit

def resblock(x, kernelsize, filters):
    fx = keras.layers.Conv2D(filters, kernelsize, activation='relu', padding='same')(x)
    fx = keras.layers.BatchNormalization()(fx)
    fx = keras.layers.Conv2D(filters, kernelsize, padding='same')(fx)
    out = keras.layers.Add()([x,fx])
    out = keras.layers.ReLU()(out)
    out = keras.layers.BatchNormalization()(out)
    return out

In [ ]:
# Mish Activation
def mish(inputs):
    x = tf.nn.softplus(inputs)
    x = tf.nn.tanh(x)
    x = tf.multiply(x,inputs)
    return x

In [ ]:
# Ranger Optimizer
# www.kaggle.com/code/yazanmajzob/ranger-optimizeranvas.com

import tensorflow_addons as tfa

def Ranger(sync_period=6,
           slow_step_size=0.5,
           learning_rate=0.001,
           beta_1=0.9,
           beta_2=0.999,
           epsilon=1e-7,
           weight_decay=0.,
           amsgrad=False,
           sma_threshold=5.0,
           total_steps=0,
           warmup_proportion=0.1,
           min_lr=0.,
           name="Ranger"):
    inner = tfa.optimizers.RectifiedAdam(learning_rate, beta_1, beta_2, epsilon, weight_decay, amsgrad, sma_threshold, total_steps, warmup_proportion, min_lr, name)
    optim = tfa.optimizers.Lookahead(inner, sync_period, slow_step_size, name)
    return optim

### Model Building

In [ ]:
def basecsiTime_block(input_layer):
    # layer_1 = Conv2D(10, (1,1), padding='same', activation='relu')(input_layer)
    layer_1 = Conv2D(32, (10,10), padding='same', activation='relu')(input_layer)
    # layer_1 = SeparableConv2D(32, (10,10), padding='same', activation='relu')(input_layer)

    # layer_2 = Conv2D(10, (1,1), padding='same', activation='relu')(input_layer)
    layer_2 = Conv2D(32, (20,20), padding='same', activation='relu')(input_layer)
    # layer_2 = SeparableConv2D(32, (20,20), padding='same', activation='relu')(input_layer)

    # layer_3 = Conv2D(10, (1,1), padding='same')(input_layer)
    layer_3 = Conv2D(32, (40,40), padding='same', activation='relu')(input_layer)
    # layer_3 = SeparableConv2D(32, (40,40), padding='same', activation='relu')(input_layer)

    layer_4 = keras.layers.MaxPooling2D(pool_size=(1,1))(input_layer)

    # layer_5 = keras.layers.concatenate([layer_1, layer_2, layer_3, layer_4], axis = 3)
    layer_5 = keras.layers.concatenate([layer_1, layer_2, layer_3, layer_4])

    layer_6 = keras.layers.BatchNormalization()(layer_5)
    output_layer = keras.layers.Dense(7, activation=mish)(layer_6)
    
    return output_layer

def csiTime_selfattn(input_layer):
    layer_1 = Conv2D(32, (10,10), padding='same', activation='relu')(input_layer)
    layer_2 = Conv2D(32, (20,20), padding='same', activation='relu')(input_layer)
    layer_3 = Conv2D(32, (40,40), padding='same', activation='relu')(input_layer)

    layer_4 = keras.layers.MaxPooling2D(pool_size=(1,1))(input_layer)
    layer_5 = keras.layers.concatenate([layer_1, layer_2, layer_3, layer_4])

    layer_6 = keras.layers.BatchNormalization()(layer_5)
    layer_7 = keras.layers.Attention()([layer_6, layer_6])
    output_layer = keras.layers.Dense(7, activation=mish)(layer_7)
    
    return output_layer

def csi_mini(input_layer):
    
    layer_1 = SeparableConv2D(32, (10,10), padding='same', activation='relu')(input_layer)
    layer_2 = SeparableConv2D(32, (20,20), padding='same', activation='relu')(input_layer)
    layer_3 = SeparableConv2D(32, (40,40), padding='same', activation='relu')(input_layer)

    layer_4 = keras.layers.MaxPooling2D(pool_size=(1,1))(input_layer)
    layer_5 = keras.layers.concatenate([layer_1, layer_2, layer_3, layer_4])

    layer_6 = keras.layers.BatchNormalization()(layer_5)
    output_layer = keras.layers.Dense(7, activation=mish)(layer_6)
    
    return output_layer

In [ ]:
from tensorflow.python.ops.nn_ops import softmax

def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape, dtype='float32')

    fir = basecsiTime_block(input_layer)
    sec = basecsiTime_block(fir)
    res1 = resblock(sec, (3,3), 7)
    thi = csiTime_selfattn(res1)
    fou = basecsiTime_block(thi)
    fif = csiTime_selfattn(fou)
    res2 = resblock(fif, (3,3), 7)
    six = basecsiTime_block(res2)

    gap = keras.layers.GlobalAveragePooling2D()(six)
    # dropout = keras.layers.Dropout(.2)(gap)
    # layer_4 = keras.layers.Flatten()(layer_4)
    output_layer = keras.layers.Dense(7, activation=softmax)(gap) #Need to change to num of classes
    
    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=X_train.shape[1:])
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.summary()

In [ ]:
epochs = 20
batch_size = 48
steps_per_epoch = X_train.shape[0]

lr_schedule = ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=100, decay_rate=0.96,
    staircase=True)

# lr_schedule = OneCycle(min_lr=7e-3, max_lr=7e-2, min_mtm = 0.85, max_mtm = 0.95, annealing_stage=0.1, annealing_rate=0.01,
#           training_iterations=(X_train.shape[0]*epochs)/(batch_size))
# lr_schedule = CosineDecayRestarts(initial_learning_rate=0.001,first_decay_steps=1000)

# clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=3e-7,
#     maximal_learning_rate=3e-2,
#     scale_fn=lambda x: 1/(2.**(x-1)),
#     step_size=2 * steps_per_epoch
# )
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "basebase.h5", save_best_only=True, monitor="val_loss"
    ),
    # OneCycle(min_lr=7e-3, max_lr=7e-2, min_mtm = 0.85, max_mtm = 0.95, annealing_stage=0.1, annealing_rate=0.01,
    #       training_iterations=(X_train.shape[0]*epochs)/(batch_size)),
    # keras.callbacks.ReduceLROnPlateau(
    #     monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    # ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, verbose=1),
]
model.compile(
    optimizer=Ranger(learning_rate = lr_schedule),
    # optimizer = keras.optimizers.SGD(lr=0.0025, momentum=0.95, nesterov=True),
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)
history = model.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
    
)

# Evaluation

In [ ]:
# model = keras.models.load_model("sa-mish-adam-150.h5")

test_loss, test_acc = model.evaluate(X_test[:,:,:], Y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
metric = "categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("Baseline's " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

In [ ]:
#Confusion Matrix and Classification Report

y_pred=model.predict(X_test) 
y_pred=np.argmax(y_pred, axis=1)
Y_test=np.argmax(Y_test, axis=1)
cm = confusion_matrix(Y_test, y_pred)
print(cm)

print('Classification Report')
print(classification_report(Y_test, y_pred, target_names=classes))

In [ ]:
%%time 

model.predict(X_test[:10])